# Bonus

## Introduction

I wanted to check something about entanglement entropy in this assignment:
are the Ising symmetry sectors of the Hamiltonian, which are uncoupled, not
entangled?
Since my code can handle building the Hamiltonian in the full x-basis as well
as permutations to and from the Ising symmetry sectors, let's test it!

We will find the ground state in the x basis and then sift out the system
into a + symmetry sector and - symmetry sector, do an svd and look at the values.

In [ ]:
import numpy as np
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
%matplotlib inline

from ph121c_lxvm import basis, tfim, tests, measure

In [ ]:
for oper_params in tests.tfim.param_sweep(
    L = [14],
    h = [1],
    bc= ['c'],
    sector=['f']
):
    job = dict(
        oper=tfim.x.H_sparse,
        oper_params=oper_params,
        solver=sla.eigsh,
        solver_params={ 
            'k' : 6, 
            'which' : 'BE',
        },
    )
    evals, evecs = tfim.data.obtain(**job)
    

In [ ]:
vals = basis.schmidt.values(
    # permute the gs so the the + and - sectors are in block forms
    evecs[basis.unitary.Ising(oper_params['L']), 0],
    # perform matricization and svd with respect to the +/- subsystems
    np.array([oper_params['L'] - 1]),
    oper_params['L']
)
print('singular values: ', vals)
print('entanglement entropy:', measure.entropy.entanglement(vals))

I'm rather confident that the code is correct because it passes all of my TFIM
consistency and basis interconsistency tests.
I might conclude that the existence of degenerate eigenspaces in the Hamiltonian
with respect to some symmetry operator does produces no entanglement entropy
across the disjoint subsystems.